In [1]:
import ompl.base as ob
import ompl.geometric as og
import ompl.util as ou
import numpy as np

In [2]:
mask = np.load('./3d_dataset_30x30x30/mask.npy')
goals = np.load('./3d_dataset_30x30x30/goals.npy')

In [3]:
Ntotal = 1000*5+200*5
ntrain = 1000*5
ntest = Ntotal-ntrain

sub = 1
Sx = int(((30 - 1) / sub) + 1)
Sy = Sx
Sz = Sx

mask_test = mask[:Ntotal][-ntest:, ::sub, ::sub, ::sub][:, :Sx, :Sy, :Sz]
mask_test = mask_test.reshape(ntest, Sx, Sy, Sz, 1)


goals_test = goals[-ntest:]
goals_test = goals_test.reshape(ntest, 3, 1)



In [5]:
average_path_cost = 0
success_count = 0

for map in range(1000):
    environment = mask_test[map,:,:,:,0].squeeze()
    space = ob.RealVectorStateSpace(3)
    bounds = ob.RealVectorBounds(3)
    bounds.setLow(0)
    bounds.setHigh(0, environment.shape[0] - 1)
    bounds.setHigh(1, environment.shape[1] - 1)
    bounds.setHigh(2, environment.shape[2] - 1)
    space.setBounds(bounds)

    start = ob.State(space)
    start[0], start[1], start[2] = 0, 0, 0

    goal = ob.State(space)
    goal[0], goal[1], goal[2] = goals_test[map,0,0], goals_test[map,1,0], goals_test[map,2,0]

    def isStateValid(state):
        x, y, z = int(state[0]), int(state[1]), int(state[2])
        if 0 <= x < environment.shape[0] and 0 <= y < environment.shape[1] and 0 <= z < environment.shape[2]:
            return environment[x, y, z] == 1
        return False

    si = ob.SpaceInformation(space)
    si.setStateValidityChecker(ob.StateValidityCheckerFn(isStateValid))

    pdef = ob.ProblemDefinition(si)
    pdef.setStartAndGoalStates(start, goal)

    planner = og.RRTstar(si)
    planner.setProblemDefinition(pdef)

    # Set the maximum edge length (range)
    max_edge_length = 1.0  # Adjust this value as needed
    planner.setRange(max_edge_length)

    planner.setup()
    solve_time = 0.030
    solved = planner.solve(solve_time)

    if solved:
        success_count += 1
        path = pdef.getSolutionPath()
        path_cost = path.cost(ob.PathLengthOptimizationObjective(si))
        average_path_cost += path_cost.value()

print(average_path_cost / success_count)


Info:    RRTstar: Space information setup was not yet called. Calling now.
Info:    RRTstar: No optimization objective specified. Defaulting to optimizing path length for the allowed planning time.
Info:    RRTstar: Started planning with 1 states. Seeking a solution better than 0.00000.
Info:    RRTstar: Initial k-nearest value of 45
Info:    RRTstar: Found an initial solution with a cost of 33.63 in 236 iterations (225 vertices in the graph)
Info:    RRTstar: Created 392 new states. Checked 76958 rewire options. 1 goal states in tree. Final solution cost 33.629
Info:    RRTstar: Space information setup was not yet called. Calling now.
Info:    RRTstar: No optimization objective specified. Defaulting to optimizing path length for the allowed planning time.
Info:    RRTstar: Started planning with 1 states. Seeking a solution better than 0.00000.
Info:    RRTstar: Initial k-nearest value of 45
Info:    RRTstar: Found an initial solution with a cost of 55.84 in 247 iterations (242 vertice